In [1]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

In [18]:
load_dotenv()

True

In [6]:
df = pd.read_csv(os.path.join(os.getcwd(),f"shakespeare_plays.csv"))

In [24]:
os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")

'text-embedding-ada-002'

In [7]:
df.rename(columns={"Unnamed: 0": "id"}, inplace=True)

In [8]:
df.head()

,id,play_name,genre,character,act,scene,sentence,text,sex
0,0,All's Well That Ends Well,Comedy,Countess,1,1,1,"In delivering my son from me, I bury a second ...",female
1,1,All's Well That Ends Well,Comedy,Bertram,1,1,2,"And I in going, madam, weep o'er my father's d...",male
2,2,All's Well That Ends Well,Comedy,Bertram,1,1,3,"anew: but I must attend his majesty's command, to",male
3,3,All's Well That Ends Well,Comedy,Bertram,1,1,4,"whom I am now in ward, evermore in subjection.",male
4,4,All's Well That Ends Well,Comedy,Lafeu,1,1,5,"You shall find of the king a husband, madam; you,",male


In [10]:
df.shape

(108093, 9)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108093 entries, 0 to 108092
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         108093 non-null  int64 
 1   play_name  108093 non-null  object
 2   genre      108093 non-null  object
 3   character  108093 non-null  object
 4   act        108093 non-null  int64 
 5   scene      108093 non-null  int64 
 6   sentence   108093 non-null  int64 
 7   text       108093 non-null  object
 8   sex        108093 non-null  object
dtypes: int64(4), object(5)
memory usage: 7.4+ MB


In [12]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["text"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
len(df)

108093

In [63]:
test_df = df.head(100_00)

In [31]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

test_df['ada_v2'] = test_df["text"].apply(
    lambda x : generate_embeddings (
        x, model = "rag-embed"
        )
    ) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

/tmp/ipykernel_31162/299800236.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['ada_v2'] = test_df["text"].apply(


In [33]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="rag-embed"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="rag-embed" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    test_df["similarities"] = test_df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

In [34]:
res = search_docs(test_df, "Can I get information on play content", top_n=4)


/tmp/ipykernel_31162/67649753.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["similarities"] = test_df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))


,id,play_name,genre,character,act,scene,sentence,text,sex,n_tokens,ada_v2,similarities
419,419,All's Well That Ends Well,Comedy,Steward,1,3,112,you something to know it.,male,6,"[0.007861711084842682, 0.0005574531387537718, ...",0.753208
273,273,All's Well That Ends Well,Comedy,King,1,2,52,Might be a copy to these younger times;,male,10,"[-0.007222725544124842, -0.011709978803992271,...",0.752989
421,421,All's Well That Ends Well,Comedy,Countess,1,3,114,yourself: many likelihoods informed me of this,female,10,"[-0.017993343994021416, -0.015138952061533928,...",0.750210
202,202,All's Well That Ends Well,Comedy,Parolles,1,1,203,counsel and understand what advice shall thrus...,male,9,"[-0.01668488420546055, -0.014798886142671108, ...",0.745782


In [64]:
test_df['text'].to_csv('main_text.txt', sep=' ', index=False)

In [65]:
from langchain_community.document_loaders import TextLoader   
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [66]:
raw_documents = TextLoader(os.path.join(os.getcwd(),f"main_text.txt")).load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [67]:
db = FAISS.from_documents(documents, AzureOpenAIEmbeddings(
            azure_deployment="rag-embed",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
            )
        )

In [51]:
type(db)

langchain_community.vectorstores.faiss.FAISS

In [68]:
query = "tell me a play with 100 words only"
docs = db.similarity_search(query)
print(docs[0].page_content)

text
"In delivering my son from me, I bury a second husband."
"And I in going, madam, weep o'er my father's death"
"anew: but I must attend his majesty's command, to"
"whom I am now in ward, evermore in subjection."
"You shall find of the king a husband, madam; you,"
"sir, a father: he that so generally is at all times"
"good must of necessity hold his virtue to you; whose"
"worthiness would stir it up where it wanted rather"
"than lack it where there is such abundance."
"What hope is there of his majesty's amendment?"
"He hath abandoned his physicians, madam; under whose"
"practises he hath persecuted time with hope, and"
"finds no other advantage in the process but only the"
"losing of hope by time."
"This young gentlewoman had a father,--O, that"
"'had'! how sad a passage 'tis!--whose skill was"
"almost as great as his honesty; had it stretched so"
"far, would have made nature immortal, and death"
"should have play for lack of work. Would, for the"
"king's sake, he were living! I th